In [6]:
import os
import subprocess
import shutil

def reduce_image_size(input_file, target_size_mb):
    # Check if ffmpeg is installed
    if not shutil.which('ffmpeg'):
        raise RuntimeError("ffmpeg is not installed. Please install it first using 'brew install ffmpeg' on Mac")
    
    # Check if input file exists
    if not os.path.exists(input_file):
        raise FileNotFoundError(f"Input file not found: {input_file}")
        
    # Convert target size from MB to bytes
    target_size_bytes = target_size_mb * 1024 * 1024
    
    # Get the directory and filename
    directory = os.path.dirname(input_file)
    filename = os.path.basename(input_file)
    name, ext = os.path.splitext(filename)
    
    # Create output filename with _reduced suffix
    output_file = os.path.join(directory, f"{name}_reduced{ext}")
    
    # Initial scale factor (1.0 = 100%)
    scale = 1.0
    
    while True:
        # Get original dimensions
        probe_cmd = ['ffprobe', '-v', 'error', '-select_streams', 'v:0', 
                    '-show_entries', 'stream=width,height', '-of', 'csv=p=0', input_file]
        result = subprocess.run(probe_cmd, capture_output=True, text=True)
        width, height = map(int, result.stdout.strip().split(','))
        
        # Calculate new dimensions
        new_width = int(width * scale)
        new_height = int(height * scale)
        
        # Construct ffmpeg command with absolute dimensions instead of percentages
        cmd = [
            'ffmpeg', '-i', input_file,
            '-vf', f'scale={new_width}:{new_height}',
            '-y',  # Overwrite output file if exists
            output_file
        ]
        
        try:
            # Run ffmpeg
            subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
            
            # Check output file size
            output_size = os.path.getsize(output_file)
            
            if output_size <= target_size_bytes:
                break
                
            # Reduce scale by 10% and try again
            scale -= 0.1
            
            # Prevent infinite loop if image can't be reduced enough
            if scale <= 0.1:
                print("Warning: Could not reduce image to target size")
                break
                
        except subprocess.CalledProcessError as e:
            raise RuntimeError(f"Error running ffmpeg: {e.stderr.decode()}")
            
    return output_file

# Example usage:
target_size_mb = 1.0  # Target size in megabytes
input_image = "/Users/shreyashgupta/Desktop/headshot copy.jpg"  # Replace with your image path

# Verify file path is correct and file exists
if not os.path.exists(input_image):
    print(f"Error: File not found at path: {input_image}")
    print("Please check if:")
    print("1. The file path is correct")
    print("2. There are no special characters in the path")
    print("3. You have necessary permissions to access the file")
else:
    reduced_image = reduce_image_size(input_image, target_size_mb)
    print(f"Reduced image saved to: {reduced_image}")


Reduced image saved to: /Users/shreyashgupta/Desktop/headshot copy_reduced.jpg
